In [ ]:
import os
import cv2
import csv
import math
import pickle
import numpy as np
import pandas as pd
import mediapipe as mp

# importing drawing tools
mp_drawing = mp.solutions.drawing_utils 

# importing the pose estimation models
mp_pose = mp.solutions.pose 

# importing media on which prediction must be made
cap = cv2.VideoCapture(0)

# counter
counter = 0
stage = None

# setting up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        
        # ret is return and frame is the video
        # if ret == false then media wasn't found
        ret, frame = cap.read()
        
        # recolor image to RGB and pass it to Mediapipe because OpenCV's default format is BGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # making detections
        results = pose.process(image)
        
        # recolor back to openCV's default format BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # rendering detections on video
        # passing three parameters - image, the landmarks and which landmark connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), # landmark/joints color
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) # connections/bones color
                                 )
        # extracting landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # extracting landmarks of hip,knee and ankle (both left and right side)
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            
            # calculating angle at both knees
            angleAtLeftKnee = round(calculate_angle(left_hip, left_knee, left_ankle),2)
            angleAtRightKnee = round(calculate_angle(right_hip, right_knee, right_ankle),2)
            meanKnee = round((angleAtLeftKnee + angleAtRightKnee)/2,2)
            
            
            # calculating angle at hip
            angleAtLeftHip = round(calculate_angle(left_shoulder, left_hip, left_knee),2)
            angleAtRightHip = round(calculate_angle(right_shoulder, right_hip, right_knee),2)
            meanHip = round((angleAtLeftHip + angleAtRightHip)/2,2)
            
            # distance between both knees
            distance = calculate_distance(right_knee,left_knee)
            
            # creating a list which has angle at left and right knee, mean of these angles and distance between them respectively
            features = [meanHip,meanKnee,distance]
            features = list(np.array(features))
            
            # Make Detections
            X = pd.DataFrame([features])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
                           
        except:
            pass
        
        # squat counter logic
        if meanKnee < 120:
            stage = "down"
        if meanKnee > 160 and stage =='down':
            stage="up"
            counter +=1
            print(counter)
        
        screen_res = 1280, 720
        scale_width = screen_res[0] / image.shape[1]
        scale_height = screen_res[1] / image.shape[0]
        scale = min(scale_width, scale_height)
        #resized window width and height
        window_width = int(image.shape[1] * scale)
        window_height = int(image.shape[0] * scale)
        #cv2.WINDOW_NORMAL makes the output window resizealbe
        cv2.namedWindow('Resized Window', cv2.WINDOW_NORMAL)
        #resize the window according to the screen resolution
        cv2.resizeWindow('Resized Window', window_width, window_height)
        
        # Get status box
        cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
        # Display Class
        cv2.putText(image, 'FORM',
                    (130,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, body_language_class.split(' ')[0],
                    (80,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Display Rep
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (25,42), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        
        cv2.imshow('Resized Window', image)
    
        
        
        # to check if tried to close our screen or we pressed the button q
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()